## Use AI Agent Tools on COCA KWIC results

In [1]:
import getout_of_text_3 as got3
import pandas as pd

In [2]:
got3.__version__

'0.3.3'

### Read local offline COCA corpus into workspace

In [3]:
coca_corpus = got3.read_corpus('../../data/coca/coca-text/')

Genres:   0%|          | 0/8 [00:00<?, ?genre/s]

Processing genre: mag


Genres:  12%|█▎        | 1/8 [00:00<00:06,  1.03genre/s]

Finished genre: mag (total files: 30)
Processing genre: web


Genres:  25%|██▌       | 2/8 [00:01<00:05,  1.03genre/s]

...

Finished genre: blog (total files: 34)
Processing genre: fic


Genres:  88%|████████▊ | 7/8 [00:06<00:00,  1.08genre/s]

Finished genre: fic (total files: 30)
Processing genre: tvm


Genres: 100%|██████████| 8/8 [00:07<00:00,  1.06genre/s]

Finished genre: tvm (total files: 30)


In [4]:
# Calculate total word count across all COCA genres and subkeys
def count_words_in_text(text):
    """Count words in a text string."""
    if not isinstance(text, str):
        return 0
    return len(text.split())

# Calculate total word count
total_word_count = 0

print("📊 Calculating total COCA corpus word count...")

for genre, subkeys in coca_corpus.items():
    for subkey, dataframe in subkeys.items():
        if isinstance(dataframe, pd.DataFrame) and 'text' in dataframe.columns:
            # Count words in all text entries for this subkey
            subkey_word_count = dataframe['text'].apply(count_words_in_text).sum()
            total_word_count += subkey_word_count

print(f"🎯 TOTAL COCA CORPUS: {total_word_count:,} words")

📊 Calculating total COCA corpus word count...
🎯 TOTAL COCA CORPUS: 1,178,812,039 words
🎯 TOTAL COCA CORPUS: 1,178,812,039 words


In [5]:
coca_corpus.keys()

dict_keys(['mag', 'web', 'acad', 'news', 'spok', 'blog', 'fic', 'tvm'])


____________________________
## Search Keyword 

- using `bovine` as a test keyword across the full COCA corpus
- COMPARE YOUR RESULTS TO THE OUTPUT HERE, IF POSSIBLE: https://www.english-corpora.org/coca/
  - I get sometimes less and sometimes more hits! TBD and needs review...


### Comparing parallel vs non-parallel kwic search

- the `n_jobs` parameter will automatically use n-1 cores to use all but one of your CPU cores. This leads to much better performance on large corpora.
- i.e. for `bovine` on the full COCA text corpus, I get (10-1=9 CPU cores):
  - non-parallel: time elapsed: 0 days 00:01:01.157718
  - parallel: time elapsed: 0 days 00:00:22.578978
  - almost 3x faster!

In [6]:
keyword='best system' # for emissions reduction, West Virginia v EPA 2021

In [8]:
before = pd.Timestamp.now()
kwic_results = got3.search_keyword_corpus(keyword, coca_corpus, 
                                            case_sensitive=False,
                                            show_context=True, 
                                            context_words=15,
                                            output='json',
                                            parallel=True)
after = pd.Timestamp.now()
print('keyword {} time elapsed:'.format(keyword), after - before)

keyword best system time elapsed: 0 days 00:00:20.524888


In [9]:
kwic_results.keys()

dict_keys(['mag_1993', 'mag_1992', 'mag_1990', 'mag_1991', 'mag_1995', 'mag_1994', 'mag_1996', 'mag_1997', 'mag_2008', 'mag_2009', 'mag_2019', 'mag_2018', 'mag_2002', 'mag_2016', 'mag_2017', 'mag_2003', 'mag_2015', 'mag_2001', 'mag_2000', 'mag_2014', 'mag_2010', 'mag_2004', 'mag_2005', 'mag_2011', 'mag_2007', 'mag_2013', 'mag_2012', 'mag_2006', 'mag_1999', 'mag_1998', 'web_13', 'web_07', 'web_06', 'web_12', 'web_04', 'web_10', 'web_11', 'web_05', 'web_29', 'web_01', 'web_15', 'web_14', 'web_28', 'web_16', 'web_02', 'web_03', 'web_17', 'web_32', 'web_26', 'web_27', 'web_33', 'web_25', 'web_31', 'web_19', 'web_18', 'web_30', 'web_24', 'web_08', 'web_20', 'web_34', 'web_21', 'web_09', 'web_23', 'web_22', 'acad_2013', 'acad_2007', 'acad_2006', 'acad_2012', 'acad_2004', 'acad_2010', 'acad_2011', 'acad_2005', 'acad_2001', 'acad_2015', 'acad_2014', 'acad_2000', 'acad_2016', 'acad_2002', 'acad_2003', 'acad_2017', 'acad_1999', 'acad_1998', 'acad_1996', 'acad_1997', 'acad_1995', 'acad_1994', 'ac

In [12]:
kwic_results['acad_2012']['292']

', Kriden tries to convince us that single party rule -- dictatorship -- is the **best system** for Tunisia . He says eyebrow-raising things like : " Arab countries are n\'t ready'

### Loading AI Agent Tools

- We are developing an AI agent using Amazon Bedrock models to analyze keyword-in-context (KWIC) results from the COCA (Contemporary Corpus of American English) corpus.

Tools include:
- `CocaForensicLinguisticsTool`: provide filtered KWIC results based on linguistic criteria.

In [13]:
# COCA Computational Forensic Linguistics Agent
# Adapted from SCOTUS analysis tools for corpus linguistics analysis

from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.chat_models import init_chat_model
from typing import Optional, Type, Dict, Any, Union, List
import json
import re
from datetime import datetime


class CocaAnalysisInput(BaseModel):
    """Input for COCA corpus linguistics analysis tool."""
    keyword: str = Field(description="The keyword/phrase to analyze from COCA KWIC results")
    results_json: Union[str, Dict[str, Any]] = Field(
        description="Pre-filtered COCA KWIC JSON results from got3.search_keyword_corpus"
    )
    analysis_focus: Optional[str] = Field(
        default="forensic_linguistics", 
        description="Analysis approach: 'forensic_linguistics', 'semantic_variation', 'register_analysis', 'diachronic', 'comparative'"
    )
    max_contexts: Optional[int] = Field(
        default=None, description="DEPRECATED: No longer used. Tool processes all provided contexts."
    )
    return_json: bool = Field(
        default=False, description="If True, return structured JSON with reasoning and findings"
    )
    extraction_strategy: str = Field(
        default="all",
        description="Text extraction: 'first', 'all', or 'raw_json'"
    )
    debug: bool = Field(default=False, description="Enable debug metrics")


class CocaForensicLinguisticsTool(BaseTool):
    """
    AI tool for computational forensic linguistics analysis of COCA KWIC results.
    
    Applies systematic data science, legal scholarship, and applied linguistics 
    methodologies to analyze keyword usage patterns across COCA genres.
    """
    name: str = "coca_forensic_analysis"
    description: str = (
        "Performs computational forensic linguistics analysis on COCA KWIC results "
        "using data science and applied linguistics methodologies."
    )
    args_schema: Type[BaseModel] = CocaAnalysisInput
    model: Any = Field(exclude=True)

    def __init__(self, model, **kwargs):
        super().__init__(**kwargs)
        self.model = model

    def _run(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "forensic_linguistics",
        max_contexts: Optional[int] = None,
        return_json: bool = False,
        extraction_strategy: str = "all",
        debug: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        try:
            return self._execute(keyword, results_json, analysis_focus, max_contexts, return_json, extraction_strategy, debug)
        except Exception as e:
            error_str = str(e)
            return f"❌ Error during COCA forensic analysis: {error_str}"

    async def _arun(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "forensic_linguistics",
        max_contexts: Optional[int] = None,
        return_json: bool = False,
        extraction_strategy: str = "all",
        debug: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        return self._run(keyword, results_json, analysis_focus, max_contexts, return_json, extraction_strategy, debug)

    def _execute(self, keyword, results_json, analysis_focus, max_contexts, return_json, extraction_strategy, debug):
        # Parse and validate input
        results_dict = self._parse_coca_results(results_json)
        stats = self._compute_coca_stats(results_dict, keyword, extraction_strategy)
        
        # Extract contexts and estimate token usage
        contexts = self._extract_contexts(results_dict, max_contexts, extraction_strategy)
        
        # Debug metrics
        if debug:
            print("✅ Reading COCA results for keyword:", keyword)
            raw_chars = len(json.dumps(results_dict))
            extracted_chars = sum(len(c) for c in contexts)
            print(f"🧪 COCA DEBUG: genre_year_keys={len(results_dict)} raw_chars={raw_chars} extracted_chars={extracted_chars} total_contexts={len(contexts)}")
            
            # Debug: Show genre distribution in ALL extracted contexts
            genre_context_counts = {}
            for context in contexts:
                if context.startswith('[') and ':' in context:
                    genre = context.split(':')[0][1:]  # Extract genre from [genre:year:filename]
                    genre_context_counts[genre] = genre_context_counts.get(genre, 0) + 1
            print(f"🎯 All extracted contexts by genre: {genre_context_counts}")
            print(f"📊 Total contexts extracted: {len(contexts)}")
        
        # Build specialized prompt and check token limits
        prompt = self._build_coca_prompt(keyword, results_dict, stats, analysis_focus, max_contexts, return_json, extraction_strategy)
        # Invoke model
        response = self.model.invoke([{"role": "user", "content": prompt}])
        content = getattr(response, 'content', str(response))
        
        if return_json:
            return self._postprocess_coca_json(content, stats)
        return content

    def _parse_coca_results(self, results_json: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        """Parse COCA results JSON - expects genre->subkey->dataframe structure."""
        if isinstance(results_json, str):
            results_dict = json.loads(results_json)
        else:
            results_dict = results_json
        
        if not isinstance(results_dict, dict):
            raise ValueError("COCA results must be a dict with genre keys")
        return results_dict

    def _extract_contexts(self, results_dict: Dict[str, Any], max_contexts: Optional[int], strategy: str) -> List[str]:
        """Extract ALL context strings from COCA results - no sampling, user controls input."""
        contexts = []
        
        # Handle COCA JSON structure: {genre_year: {filename_id: text_string}}
        for genre_year_key, filename_dict in results_dict.items():
            if not isinstance(filename_dict, dict):
                continue
            
            # Split genre_year for labeling
            parts = genre_year_key.split('_')
            genre = parts[0] if len(parts) >= 1 else 'unknown'
            year = parts[1] if len(parts) >= 2 else 'unknown'
            
            # Extract ALL text content from filename_id -> text_string mappings
            for filename_id, text_content in filename_dict.items():
                if isinstance(text_content, str) and text_content.strip():
                    # Format: [genre:year:filename_id] text_content
                    context_label = f"[{genre}:{year}:{filename_id}]"
                    contexts.append(f"{context_label} {text_content.strip()}")
                        
        return contexts

    def _extract_context_from_row(self, row, strategy: str) -> str:
        """Extract context from a single COCA result row."""
        text_fields = ['context', 'text', 'kwic', 'content', 'snippet']
        
        if strategy == 'first':
            for field in text_fields:
                if hasattr(row, field) and isinstance(getattr(row, field), str):
                    return getattr(row, field).strip()
        elif strategy == 'all':
            parts = []
            for field in text_fields:
                if hasattr(row, field) and isinstance(getattr(row, field), str):
                    parts.append(getattr(row, field).strip())
            return ' | '.join(parts) if parts else ''
        
        return str(row) if strategy == 'raw_json' else ''

    def _extract_context_from_item(self, item, strategy: str) -> str:
        """Extract context from a dict/object item."""
        if isinstance(item, str):
            return item
        elif isinstance(item, dict):
            text_fields = ['context', 'text', 'kwic', 'content', 'snippet']
            if strategy == 'first':
                for field in text_fields:
                    if field in item and isinstance(item[field], str):
                        return item[field].strip()
            elif strategy == 'all':
                parts = []
                for field in text_fields:
                    if field in item and isinstance(item[field], str):
                        parts.append(item[field].strip())
                return ' | '.join(parts) if parts else ''
            elif strategy == 'raw_json':
                return json.dumps(item)
        return str(item)

    def _compute_coca_stats(self, results_dict: Dict[str, Any], keyword: str, strategy: str) -> Dict[str, Any]:
        """Compute statistics about COCA results distribution - handles {genre_year: {filename_id: text_string}} structure."""
        # Extract genres from genre_year keys
        genres = set()
        genre_counts = {}
        total_contexts = 0
        
        for genre_year_key, filename_dict in results_dict.items():
            if isinstance(filename_dict, dict):
                # Split to get genre
                parts = genre_year_key.split('_')
                if len(parts) >= 1:
                    genre = parts[0]
                    genres.add(genre)
                    
                    # Count contexts (filename entries)
                    context_count = len(filename_dict)
                    total_contexts += context_count
                    
                    # Aggregate by genre
                    if genre not in genre_counts:
                        genre_counts[genre] = 0
                    genre_counts[genre] += context_count
        
        return {
            'keyword': keyword,
            'genres': sorted(list(genres)),
            'genre_counts': genre_counts,
            'total_contexts': total_contexts,
            'extraction_strategy': strategy
        }

    def _build_coca_prompt(self, keyword: str, results_dict: Dict[str, Any], stats: Dict[str, Any], 
                          analysis_focus: str, max_contexts: Optional[int], return_json: bool, 
                          extraction_strategy: str) -> str:
        """Build specialized prompt for COCA forensic linguistics analysis."""
        
        contexts = self._extract_contexts(results_dict, max_contexts, extraction_strategy)
        
        # Build genre summary from actual stats, not just what's in contexts
        genre_summary = ", ".join([f"{g}({stats['genre_counts'][g]})" for g in stats['genres']])
        
        # Add explicit instruction about complete data inclusion
        contexts_section = f"""COCA KWIC Contexts (ALL {len(contexts)} contexts from provided data):
---
IMPORTANT: ALL contexts from your provided COCA data are included below: {genre_summary}
Each context is labeled [genre:year:filename_id] to show its source.
No sampling or filtering was performed - this is your complete dataset.
---
""" + "\n".join(contexts) + "\n---\n"
        
        focus_instructions = {
            "forensic_linguistics": """
            As a computational forensic linguist, perform systematic analysis to identify:
            1. **Semantic Range Mapping**: Document all distinct senses/meanings of the keyword
            2. **Register Variation**: Compare usage patterns across genres (academic, news, fiction, etc.)
            3. **Collocational Profiles**: Identify key collocates and their significance
            4. **Frequency Distributions**: Analyze genre-specific frequency patterns
            5. **Interpretive Stability**: Assess semantic consistency vs. context-dependency
            6. **Forensic Implications**: Note patterns relevant to authorship, text dating, or authenticity
            """,
            "semantic_variation": """
            Focus on semantic analysis:
            1. Identify polysemy patterns and meaning boundaries
            2. Map semantic fields and conceptual domains
            3. Analyze metaphorical vs. literal usage
            4. Document semantic change indicators across contexts
            """,
            "register_analysis": """
            Perform register-specific analysis:
            1. Compare formal vs. informal usage patterns
            2. Identify genre-specific conventions
            3. Analyze technical vs. general usage
            4. Map sociolinguistic variation patterns
            """,
            "diachronic": """
            Analyze temporal patterns:
            1. Identify usage evolution across time periods
            2. Map emerging vs. declining meanings
            3. Track semantic change trajectories
            4. Document historical usage patterns
            """,
            "comparative": """
            Perform comparative analysis:
            1. Cross-genre pattern comparison
            2. Usage frequency analysis
            3. Contextual distribution mapping
            4. Identify genre-specific markers
            """
        }
        
        base_prompt = f"""
        You are a computational forensic linguistics AI agent analyzing COCA (Contemporary Corpus of American English) data.

        METHODOLOGICAL FRAMEWORK:
        Apply systematic data science, legal scholarship, and applied linguistics approaches to analyze the keyword "{keyword}".

        CORPUS DATA SUMMARY:
        - Keyword: "{keyword}"
        - Total Contexts Provided: {stats['total_contexts']:,} across {len(results_dict)} genre_year combinations
        - Genre Distribution: {genre_summary}
        - Contexts Analyzed: ALL {len(contexts)} contexts (complete dataset, no sampling)
        - Extraction Strategy: {extraction_strategy}
        
        ANALYSIS FOCUS: {analysis_focus}
        {focus_instructions.get(analysis_focus, focus_instructions['forensic_linguistics'])}

        SYSTEMATIC STEPS:
        1. **Data Overview**: Summarize distribution across ALL genres (use the counts provided above)
        2. **Pattern Recognition**: Identify recurring usage patterns across different genres
        3. **Statistical Analysis**: Note frequency and distribution patterns across ALL genres
        4. **Linguistic Analysis**: Analyze syntactic, semantic, and pragmatic features by genre
        5. **Forensic Assessment**: Evaluate evidential value for text analysis
        6. **Interpretive Framework**: Provide systematic interpretation guidelines

        CRITICAL CONSTRAINTS:
        - Use ALL the provided COCA contexts (complete dataset as provided by user)
        - Apply rigorous linguistic methodology across all provided contexts
        - Avoid speculation beyond evidence
        - Maintain scientific objectivity
        - Analyze the complete distribution of contexts as provided (no sampling performed)

        {contexts_section}
        """
        
        if return_json:
            base_prompt += """
            Return ONLY valid JSON with this structure:
            {
              "keyword": string,
              "total_contexts": number,
              "genre_distribution": object,
              "reasoning_content": [string, ...],
              "semantic_analysis": string,
              "register_patterns": string,
              "forensic_implications": string,
              "summary": string,
              "limitations": string
            }
            """
        else:
            base_prompt += """
            Provide structured analysis with these sections:
            1. **Corpus Distribution Overview** (use the full genre counts provided)
            2. **Semantic Analysis** 
            3. **Register and Genre Patterns** (analyze patterns across ALL genres)
            4. **Collocational Analysis**
            5. **Forensic Linguistics Assessment**
            6. **Interpretive Guidelines**
            7. **Methodological Limitations**
            """
        
        return base_prompt.strip()

    def _postprocess_coca_json(self, content: str, stats: Dict[str, Any]) -> Dict[str, Any]:
        """Process and validate JSON response from model."""
        try:
            parsed = json.loads(content)
        except Exception:
            # Try to extract JSON from response
            match = re.search(r'{[\s\S]*}', content)
            if match:
                try:
                    parsed = json.loads(match.group(0))
                except Exception:
                    parsed = None
            else:
                parsed = None
        
        if not isinstance(parsed, dict):
            # Fallback structure
            parsed = {
                "keyword": stats['keyword'],
                "total_contexts": stats['total_contexts'],
                "genre_distribution": stats['genre_counts'],
                "reasoning_content": [
                    "Model did not return valid JSON; content auto-wrapped.",
                    "Analysis limited by response format issues."
                ],
                "semantic_analysis": content if isinstance(content, str) else str(content),
                "register_patterns": "Unable to extract due to format issues.",
                "forensic_implications": "Analysis inconclusive due to response parsing failure.",
                "summary": "Response required manual wrapping - review raw content.",
                "limitations": "Auto-wrapped due to invalid JSON from model."
            }
        
        # Ensure required fields exist
        required_fields = {
            "reasoning_content": [],
            "semantic_analysis": "",
            "register_patterns": "",
            "forensic_implications": "",
            "summary": "",
            "limitations": ""
        }
        
        for field, default in required_fields.items():
            if field not in parsed:
                parsed[field] = default
        
        return parsed
    
# Markdown export function for COCA analysis
def export_coca_markdown(result, keyword: str, filename: str = None):
    """Export COCA forensic linguistics analysis to markdown with reasoning first."""
    import json
    from datetime import datetime
    
    def _sanitize(name: str) -> str:
        return ''.join(c if (c.isalnum() or c in ('-','_')) else '_' for c in name.strip()) or 'analysis'
    
    safe_keyword = _sanitize(keyword)
    outname = filename or f"coca_forensic_{safe_keyword}.md"
    
    lines = [f"# COCA Forensic Linguistics Analysis: {keyword}\n\n"]
    lines.append(f"*Generated: {datetime.utcnow().isoformat()}Z*\n\n")
    
    if isinstance(result, dict):
        # Extract reasoning content first
        reasoning = result.get('reasoning_content', [])
        if reasoning:
            lines.append("## Methodological Framework\n\n")
            lines.append("```text\n")
            if isinstance(reasoning, list):
                lines.append('\n'.join(str(r) for r in reasoning))
            else:
                lines.append(str(reasoning))
            lines.append("\n```\n\n")
        
        # Add structured sections
        sections = [
            ('semantic_analysis', 'Semantic Analysis'),
            ('register_patterns', 'Register and Genre Patterns'),
            ('forensic_implications', 'Forensic Linguistics Assessment'),
            ('summary', 'Summary'),
            ('limitations', 'Limitations')
        ]
        
        for field, title in sections:
            if field in result and result[field]:
                lines.append(f"## {title}\n\n")
                lines.append(f"{result[field]}\n\n")
        
        # Add distribution data if available
        if 'genre_distribution' in result:
            lines.append("## Corpus Distribution\n\n")
            lines.append("```json\n")
            lines.append(json.dumps(result['genre_distribution'], indent=2))
            lines.append("\n```\n\n")
    
    else:
        lines.append("## Analysis\n\n")
        lines.append(str(result))
    
    content = ''.join(lines)
    
    with open(outname, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 COCA forensic analysis exported: {outname} ({len(content)} chars)")
    return outname

print("✅ COCA Forensic Linguistics Tool loaded!")

✅ COCA Forensic Linguistics Tool loaded!


/Users/ejacquot/Documents/Github/getout_of_text_3/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Setup AWS Bedrock Model for COCA Analysis

Initialize the same GPT-OSS-120B model used for SCOTUS analysis, but now tailored for computational forensic linguistics on COCA data.

In [14]:
# Initialize AWS Bedrock model for COCA forensic linguistics analysis
model_id = 'openai.gpt-oss-120b-1:0'  # 128K context window
max_tokens = 128000

model = init_chat_model(
    model_id, 
    model_provider="bedrock_converse",
    credentials_profile_name='atn-developer',  # Adjust to your AWS profile
    max_tokens=max_tokens
)

# Initialize the COCA forensic linguistics tool
coca_forensic_tool = CocaForensicLinguisticsTool(model=model)

print(f"✅ AWS Bedrock model initialized: {model_id}")
print(f"🔬 COCA Forensic Linguistics Tool ready")
print(f"📊 Available COCA genres: {list(coca_corpus.keys())}")

✅ AWS Bedrock model initialized: openai.gpt-oss-120b-1:0
🔬 COCA Forensic Linguistics Tool ready
📊 Available COCA genres: ['mag', 'web', 'acad', 'news', 'spok', 'blog', 'fic', 'tvm']


## COCA Forensic Linguistics Analysis Demo

Let's demonstrate the computational forensic linguistics approach on COCA data using a test keyword.

In [15]:
# First, ensure we have JSON data for analysis (not just print output)
#keyword = "gabagool"
keyword="best system"
test_keyword=keyword
context_window=20
max_tokens=128000
ratio_scale_back=1

print(f"🔍 Running COCA KWIC search for JSON analysis: '{keyword}'")

# Re-run the search with output='json' to get structured data for AI analysis
bovine_kwic_json = got3.search_keyword_corpus(
    keyword, 
    coca_corpus,
    case_sensitive=False,
    show_context=True, 
    context_words=int(context_window * ratio_scale_back),
    output='json',  # This is key - we need JSON output for AI analysis
    parallel=True
)

print(f"📊 KWIC JSON search complete!")
print(f"🔍 Found data in genres: {list(bovine_kwic_json.keys())}")

🔍 Running COCA KWIC search for JSON analysis: 'best system'
📊 KWIC JSON search complete!
🔍 Found data in genres: ['mag_1993', 'mag_1992', 'mag_1990', 'mag_1991', 'mag_1995', 'mag_1994', 'mag_1996', 'mag_1997', 'mag_2008', 'mag_2009', 'mag_2019', 'mag_2018', 'mag_2002', 'mag_2016', 'mag_2017', 'mag_2003', 'mag_2015', 'mag_2001', 'mag_2000', 'mag_2014', 'mag_2010', 'mag_2004', 'mag_2005', 'mag_2011', 'mag_2007', 'mag_2013', 'mag_2012', 'mag_2006', 'mag_1999', 'mag_1998', 'web_13', 'web_07', 'web_06', 'web_12', 'web_04', 'web_10', 'web_11', 'web_05', 'web_29', 'web_01', 'web_15', 'web_14', 'web_28', 'web_16', 'web_02', 'web_03', 'web_17', 'web_32', 'web_26', 'web_27', 'web_33', 'web_25', 'web_31', 'web_19', 'web_18', 'web_30', 'web_24', 'web_08', 'web_20', 'web_34', 'web_21', 'web_09', 'web_23', 'web_22', 'acad_2013', 'acad_2007', 'acad_2006', 'acad_2012', 'acad_2004', 'acad_2010', 'acad_2011', 'acad_2005', 'acad_2001', 'acad_2015', 'acad_2014', 'acad_2000', 'acad_2016', 'acad_2002', 'aca

In [17]:
# Debug: Show all genre_year keys returned by the search
print(f"📋 All genre_year keys found for '{keyword}':")
print("=" * 60)

sorted_keys = sorted(bovine_kwic_json.keys())
for i, key in enumerate(sorted_keys, 1):
    hit_count = len(bovine_kwic_json[key]) if isinstance(bovine_kwic_json[key], dict) else 0
    parts = key.split('_')
    genre = parts[0] if len(parts) >= 1 else 'unknown'
    year = parts[1] if len(parts) >= 2 else 'unknown'
    print(f"  {i:2}. {key} -> {hit_count:,} hits (genre: {genre}, year: {year})")

print(f"\n🎯 Total genre_year combinations: {len(sorted_keys)}")

# Show genre and year diversity
genres = set()
years = set()
for key in sorted_keys:
    parts = key.split('_')
    if len(parts) >= 2:
        genres.add(parts[0])
        years.add(parts[1])

print(f"📊 Unique genres found: {sorted(genres)}")
print(f"📅 Unique years found: {sorted(years)}")

📋 All genre_year keys found for 'best system':
   1. acad_1990 -> 0 hits (genre: acad, year: 1990)
   2. acad_1991 -> 0 hits (genre: acad, year: 1991)
   3. acad_1992 -> 1 hits (genre: acad, year: 1992)
   4. acad_1993 -> 2 hits (genre: acad, year: 1993)
   5. acad_1994 -> 0 hits (genre: acad, year: 1994)
   6. acad_1995 -> 1 hits (genre: acad, year: 1995)
   7. acad_1996 -> 0 hits (genre: acad, year: 1996)
   8. acad_1997 -> 0 hits (genre: acad, year: 1997)
   9. acad_1998 -> 0 hits (genre: acad, year: 1998)
  10. acad_1999 -> 1 hits (genre: acad, year: 1999)
  11. acad_2000 -> 0 hits (genre: acad, year: 2000)
  12. acad_2001 -> 0 hits (genre: acad, year: 2001)
  13. acad_2002 -> 0 hits (genre: acad, year: 2002)
  14. acad_2003 -> 2 hits (genre: acad, year: 2003)
  15. acad_2004 -> 0 hits (genre: acad, year: 2004)
  16. acad_2005 -> 0 hits (genre: acad, year: 2005)
  17. acad_2006 -> 1 hits (genre: acad, year: 2006)
  18. acad_2007 -> 1 hits (genre: acad, year: 2007)
  19. acad_2008 -

In [22]:
import random
import pandas as pd

# Comprehensive COCA bovine data analysis
print("📊 COCA '{}' Distribution Analysis".format(keyword))
print("=" * 50)

# Analyze the structure: {genre_year: {filename_id: text_string}}
genre_year_counts = {}
total_hits = 0

# Count hits per genre_year combination
for genre_year_key, filename_dict in bovine_kwic_json.items():
    # Count the number of filename entries (each represents a hit)
    hit_count = len(filename_dict) if isinstance(filename_dict, dict) else 0
    genre_year_counts[genre_year_key] = hit_count
    total_hits += hit_count

print(f"🎯 Total hits: {total_hits:,}")
print(f"📋 Total genre_year combinations: {len(genre_year_counts)}")

# 1. GENRE TOTALS (split on '_' and aggregate by genre [0])
print(f"\n📈 Hits by GENRE (total across all years):")
genre_totals = {}
for genre_year_key, hit_count in genre_year_counts.items():
    # Split on '_' and take first part as genre
    parts = genre_year_key.split('_')
    if len(parts) >= 2:
        genre = parts[0]  # First part is genre
        if genre not in genre_totals:
            genre_totals[genre] = 0
        genre_totals[genre] += hit_count

for genre, total in sorted(genre_totals.items(), key=lambda x: x[1], reverse=True):
    percentage = (total / total_hits * 100) if total_hits > 0 else 0
    print(f"  {genre}: {total:,} hits ({percentage:.1f}%)")

# 2. YEAR TOTALS (split on '_' and aggregate by year [1])
print(f"\n📅 Hits by YEAR (total across all genres):")
year_totals = {}
for genre_year_key, hit_count in genre_year_counts.items():
    # Split on '_' and take second part as year
    parts = genre_year_key.split('_')
    if len(parts) >= 2:
        year = parts[1]  # Second part is year
        if year not in year_totals:
            year_totals[year] = 0
        year_totals[year] += hit_count

# Sort years by hit count (top 15)
for year, total in sorted(year_totals.items(), key=lambda x: x[1], reverse=True)[:15]:
    percentage = (total / total_hits * 100) if total_hits > 0 else 0
    print(f"  {year}: {total:,} hits ({percentage:.1f}%)")

# 3. TOP 10 GENRE_YEAR combinations by hit count
print(f"\n🔥 Top 10 genre_year combinations by hit count:")
sorted_counts = sorted(genre_year_counts.items(), key=lambda x: x[1], reverse=True)
for i, (genre_year_key, count) in enumerate(sorted_counts[:10], 1):
    parts = genre_year_key.split('_')
    genre = parts[0] if len(parts) >= 1 else 'unknown'
    year = parts[1] if len(parts) >= 2 else 'unknown'
    print(f"  {i:2}. {genre_year_key}: {count:,} hits (genre: {genre}, year: {year})")

# 4. Random example for inspection
print(f"\n🎲 Random Example Context:")
print("=" * 30)

# Only select from genre_year combinations that have actual data
non_empty_keys = [key for key, filename_dict in bovine_kwic_json.items() 
                  if isinstance(filename_dict, dict) and len(filename_dict) > 0]

if non_empty_keys:
    random_genre_year = random.choice(non_empty_keys)
    random_filename_id = random.choice(list(bovine_kwic_json[random_genre_year].keys()))
    random_text = bovine_kwic_json[random_genre_year][random_filename_id]

    parts = random_genre_year.split('_')
    genre = parts[0] if len(parts) >= 1 else 'unknown'
    year = parts[1] if len(parts) >= 2 else 'unknown'

    print(f"Genre_Year Key: {random_genre_year}")
    print(f"  - Genre: {genre}")
    print(f"  - Year: {year}")
    print(f"Filename ID: {random_filename_id}")
    print(f"Total hits in this genre_year: {genre_year_counts.get(random_genre_year, 0):,}")
    print(f"\nSample text content:")
    print(f"{str(random_text)[:200]}..." if len(str(random_text)) > 200 else str(random_text))
else:
    print("❌ No genre_year combinations contain actual hit data")

📊 COCA 'best system' Distribution Analysis
🎯 Total hits: 140
📋 Total genre_year combinations: 248

📈 Hits by GENRE (total across all years):
  blog: 31 hits (22.1%)
  spok: 23 hits (16.4%)
  web: 21 hits (15.0%)
  mag: 20 hits (14.3%)
  news: 20 hits (14.3%)
  acad: 15 hits (10.7%)
  tvm: 6 hits (4.3%)
  fic: 4 hits (2.9%)

📅 Hits by YEAR (total across all genres):
  1993: 6 hits (4.3%)
  1991: 6 hits (4.3%)
  1992: 5 hits (3.6%)
  2009: 5 hits (3.6%)
  15: 5 hits (3.6%)
  1995: 4 hits (2.9%)
  1994: 4 hits (2.9%)
  2003: 4 hits (2.9%)
  2000: 4 hits (2.9%)
  2014: 4 hits (2.9%)
  2011: 4 hits (2.9%)
  2012: 4 hits (2.9%)
  17: 4 hits (2.9%)
  1990: 3 hits (2.1%)
  2002: 3 hits (2.1%)

🔥 Top 10 genre_year combinations by hit count:
   1. blog_17: 4 hits (genre: blog, year: 17)
   2. news_1991: 3 hits (genre: news, year: 1991)
   3. news_2005: 3 hits (genre: news, year: 2005)
   4. spok_2009: 3 hits (genre: spok, year: 2009)
   5. spok_2000: 3 hits (genre: spok, year: 2000)
   6. blog_1

In [23]:
print(f"🔍 Found data in genres: {list(bovine_kwic_json.keys())}")

🔍 Found data in genres: ['mag_1993', 'mag_1992', 'mag_1990', 'mag_1991', 'mag_1995', 'mag_1994', 'mag_1996', 'mag_1997', 'mag_2008', 'mag_2009', 'mag_2019', 'mag_2018', 'mag_2002', 'mag_2016', 'mag_2017', 'mag_2003', 'mag_2015', 'mag_2001', 'mag_2000', 'mag_2014', 'mag_2010', 'mag_2004', 'mag_2005', 'mag_2011', 'mag_2007', 'mag_2013', 'mag_2012', 'mag_2006', 'mag_1999', 'mag_1998', 'web_13', 'web_07', 'web_06', 'web_12', 'web_04', 'web_10', 'web_11', 'web_05', 'web_29', 'web_01', 'web_15', 'web_14', 'web_28', 'web_16', 'web_02', 'web_03', 'web_17', 'web_32', 'web_26', 'web_27', 'web_33', 'web_25', 'web_31', 'web_19', 'web_18', 'web_30', 'web_24', 'web_08', 'web_20', 'web_34', 'web_21', 'web_09', 'web_23', 'web_22', 'acad_2013', 'acad_2007', 'acad_2006', 'acad_2012', 'acad_2004', 'acad_2010', 'acad_2011', 'acad_2005', 'acad_2001', 'acad_2015', 'acad_2014', 'acad_2000', 'acad_2016', 'acad_2002', 'acad_2003', 'acad_2017', 'acad_1999', 'acad_1998', 'acad_1996', 'acad_1997', 'acad_1995', 'a

In [26]:
analysis_result = coca_forensic_tool._run(
    keyword=keyword,
    results_json=bovine_kwic_json,  # Use the JSON data
    analysis_focus="forensic_linguistics",
    #max_contexts=50,  # Limit for demo
    return_json=False,
    extraction_strategy="all",
    debug=True
)

✅ Reading COCA results for keyword: best system
🧪 COCA DEBUG: genre_year_keys=248 raw_chars=34866 extracted_chars=31593 total_contexts=140
🎯 All extracted contexts by genre: {'mag': 20, 'web': 21, 'acad': 15, 'news': 20, 'spok': 23, 'blog': 31, 'fic': 4, 'tvm': 6}
📊 Total contexts extracted: 140


In [34]:
analysis_result

[{'type': 'reasoning_content',
  'reasoning_content': {'text': 'We need to produce structured analysis based on provided contexts. We must summarize counts, semantic range, register variation, collocates, frequency patterns, forensic implications, guidelines, limitations.\n\nWe have 140 contexts across 8 genres with counts: acad15, blog31, fic4, mag20, news20, spok23, tvm6, web21. Provide overview.\n\nSemantic analysis: Identify senses: (1) superlative claim about system being optimal (e.g., political/economic system, healthcare, education) (2) technical/engineering sense referring to actual mechanical system (best system for oil, a setup, hardware) (3) evaluative marketing/promotion of product (best system for organizing music, best system for solar electricity) (4) rhetorical hyperbole praising something as the "best system" in discourse (e.g., "best system ever").\n\nRegister patterns: academic contexts discuss selection of best system, education, emission reduction, etc.; news ofte

In [35]:
export_coca_markdown_blocks(analysis_result, keyword)

📄 Enhanced COCA forensic analysis exported: coca_forensic_best_system_blocks.md (19207 chars)


'coca_forensic_best_system_blocks.md'

### For a keyword like `vehicle` where there are MANY hits in COCA, strategies include:

- filter on a smaller context window
- randomly sample from the full set of hits, as to preserve all genre_year combos with at least one hit but then cull others (which could lead to over-representation of certain genre_year combos with few hits, but for the sake of demo, this is acceptable)

In [ ]:
sample_down_ratio=0.3

len(bovine_kwic_json)
bovine_kwic_json.keys()
# for each key, let's filter out 30% of the data randomly to reduce input size
import random
reduced_bovine_kwic_json = {}
for genre_year_key, filename_dict in bovine_kwic_json.items():
    if isinstance(filename_dict, dict):
        filenames = list(filename_dict.keys())
        sample_size = max(1, int(len(filenames) * 0.3))  # Ensure at least one entry
        sampled_filenames = random.sample(filenames, sample_size)
        reduced_bovine_kwic_json[genre_year_key] = {fn: filename_dict[fn] for fn in sampled_filenames}
    else:
        reduced_bovine_kwic_json[genre_year_key] = filename_dict

ValueError: Sample larger than population or is negative

In [21]:
print(f"\n🔬 Running COCA forensic linguistics analysis...")
    
analysis_result = coca_forensic_tool._run(
    keyword=test_keyword,
    results_json=reduced_bovine_kwic_json,  # Use the JSON data
    analysis_focus="forensic_linguistics",
    #max_contexts=50,  # Limit for demo
    return_json=False,
    extraction_strategy="all",
    debug=True
)

print(f"\n✅ Analysis complete!")
print(f"Result type: {type(analysis_result)}")

if isinstance(analysis_result, str):
    print(f"\n{analysis_result[:]}...")
elif isinstance(analysis_result, list):
    print(f"Got {len(analysis_result)} result blocks")
    for i, block in enumerate(analysis_result[:2]):  # Show first 2 blocks
        print(f"Block {i+1}: {type(block)} - {str(block)[:100]}...")


🔬 Running COCA forensic linguistics analysis...
✅ Reading COCA results for keyword: best system
🧪 COCA DEBUG: genre_year_keys=2 raw_chars=699 extracted_chars=681 total_contexts=3
🎯 All extracted contexts by genre: {'mag': 3}
📊 Total contexts extracted: 3


KeyboardInterrupt: 

In [20]:
# Function to calculate optimal sample ratio from AWS Bedrock token overage
def calculate_optimal_ratio(current_ratio, token_overage, max_tokens=128000):
    """
    Calculate the optimal sampling ratio to get as close as possible to max token count.
    
    Args:
        current_ratio: The ratio that caused the overage (e.g., 0.3)
        token_overage: Positive number of tokens over the limit (e.g., 8010)
        max_tokens: Model's maximum token limit (default 128000)
    
    Returns:
        optimal_ratio: Suggested ratio to use
        estimated_tokens: Expected token count with optimal ratio
    """
    # Current estimated tokens = max_tokens + overage
    current_estimated_tokens = max_tokens + token_overage
    
    # Calculate the ratio needed to fit within max_tokens
    # We want: current_estimated_tokens * scale_factor = max_tokens
    scale_factor = max_tokens / current_estimated_tokens
    
    # Apply scale factor to current ratio
    optimal_ratio = current_ratio * scale_factor
    
    # Add small buffer (reduce by 5%) to ensure we stay under limit
    optimal_ratio_with_buffer = optimal_ratio * 0.95
    
    estimated_tokens = current_estimated_tokens * scale_factor * 0.95
    
    return optimal_ratio_with_buffer, estimated_tokens

def extract_token_overage_from_error(error_str):
    """Extract token overage from AWS Bedrock error message."""
    import re
    match = re.search(r'got (-?\d+)', error_str)
    if match:
        negative_tokens = int(match.group(1))
        return abs(negative_tokens)  # Convert negative to positive overage
    return None

def auto_calculate_ratio_from_error(error_str, current_ratio):
    """Automatically calculate optimal ratio from AWS Bedrock error."""
    overage = extract_token_overage_from_error(error_str)
    if overage:
        optimal, estimated = calculate_optimal_ratio(current_ratio, overage)
        return {
            'current_ratio': current_ratio,
            'token_overage': overage,
            'optimal_ratio': optimal,
            'estimated_tokens': estimated,
            'utilization_percent': estimated/128000*100
        }
    return None

# Example calculations based on your data
print("🧮 Calculating optimal ratios from your AWS Bedrock errors:")
print("=" * 60)

examples = [
    (1.0, 340508),
    (0.5, 103739), 
    (0.3, 8010)
]

for ratio, overage in examples:
    optimal, estimated = calculate_optimal_ratio(ratio, overage)
    print(f"Ratio {ratio:.2f} → overage {overage:,} tokens")
    print(f"  → Optimal ratio: {optimal:.4f}")
    print(f"  → Estimated tokens: {estimated:,.0f}")
    print(f"  → Token utilization: {estimated/128000*100:.1f}%")
    print()

# Quick calculator for your current case
current_ratio = 0.3
current_overage = 8010
optimal, estimated = calculate_optimal_ratio(current_ratio, current_overage)

print(f"🎯 For your current case:")
print(f"Current ratio: {current_ratio}")
print(f"Current overage: {current_overage:,} tokens")
print(f"Optimal ratio: {optimal:.4f}")
print(f"Expected tokens: {estimated:,.0f} / 128,000 ({estimated/128000*100:.1f}%)")
print(f"Since 0.27 worked, the calculation is accurate! ({optimal:.4f} ≈ 0.27)")

# Test the error parsing function
test_errors = [
    "max_tokens must be at least 1, got -340508.",
    "max_tokens must be at least 1, got -103739.", 
    "max_tokens must be at least 1, got -8010."
]

print(f"\n🔧 Testing automatic error parsing:")
for i, error in enumerate(test_errors):
    ratio = [1.0, 0.5, 0.3][i]
    result = auto_calculate_ratio_from_error(error, ratio)
    if result:
        print(f"Error: {error}")
        print(f"  → Auto-calculated optimal ratio: {result['optimal_ratio']:.4f}")
        print(f"  → Expected utilization: {result['utilization_percent']:.1f}%")# Function to calculate optimal sample ratio from AWS Bedrock token overage
def calculate_optimal_ratio(current_ratio, token_overage, max_tokens=128000):
    """
    Calculate the optimal sampling ratio to get as close as possible to max token count.
    
    Args:
        current_ratio: The ratio that caused the overage (e.g., 0.3)
        token_overage: Positive number of tokens over the limit (e.g., 8010)
        max_tokens: Model's maximum token limit (default 128000)
    
    Returns:
        optimal_ratio: Suggested ratio to use
        estimated_tokens: Expected token count with optimal ratio
    """
    # Current estimated tokens = max_tokens + overage
    current_estimated_tokens = max_tokens + token_overage
    
    # Calculate the ratio needed to fit within max_tokens
    # We want: current_estimated_tokens * scale_factor = max_tokens
    scale_factor = max_tokens / current_estimated_tokens
    
    # Apply scale factor to current ratio
    optimal_ratio = current_ratio * scale_factor
    
    # Add small buffer (reduce by 5%) to ensure we stay under limit
    optimal_ratio_with_buffer = optimal_ratio * 0.95
    
    estimated_tokens = current_estimated_tokens * scale_factor * 0.95
    
    return optimal_ratio_with_buffer, estimated_tokens

# Example calculations based on your data
print("🧮 Calculating optimal ratios from your AWS Bedrock errors:")
print("=" * 60)

examples = [
    (1.0, 340508),
    (0.5, 103739), 
    (0.3, 8010)
]

for ratio, overage in examples:
    optimal, estimated = calculate_optimal_ratio(ratio, overage)
    print(f"Ratio {ratio:.2f} → overage {overage:,} tokens")
    print(f"  → Optimal ratio: {optimal:.4f}")
    print(f"  → Estimated tokens: {estimated:,.0f}")
    print(f"  → Token utilization: {estimated/128000*100:.1f}%")
    print()

# Quick calculator for your current case
current_ratio = 0.3
current_overage = 8010
optimal, estimated = calculate_optimal_ratio(current_ratio, current_overage)

print(f"🎯 For your current case:")
print(f"Current ratio: {current_ratio}")
print(f"Current overage: {current_overage:,} tokens")
print(f"Optimal ratio: {optimal:.4f}")
print(f"Expected tokens: {estimated:,.0f} / 128,000 ({estimated/128000*100:.1f}%)")
print(f"Since 0.27 worked, the calculation is accurate! ({optimal:.4f} ≈ 0.27)")

🧮 Calculating optimal ratios from your AWS Bedrock errors:
Ratio 1.00 → overage 340,508 tokens
  → Optimal ratio: 0.2595
  → Estimated tokens: 121,600
  → Token utilization: 95.0%

Ratio 0.50 → overage 103,739 tokens
  → Optimal ratio: 0.2624
  → Estimated tokens: 121,600
  → Token utilization: 95.0%

Ratio 0.30 → overage 8,010 tokens
  → Optimal ratio: 0.2682
  → Estimated tokens: 121,600
  → Token utilization: 95.0%

🎯 For your current case:
Current ratio: 0.3
Current overage: 8,010 tokens
Optimal ratio: 0.2682
Expected tokens: 121,600 / 128,000 (95.0%)
Since 0.27 worked, the calculation is accurate! (0.2682 ≈ 0.27)

🔧 Testing automatic error parsing:
Error: max_tokens must be at least 1, got -340508.
  → Auto-calculated optimal ratio: 0.2595
  → Expected utilization: 95.0%
Error: max_tokens must be at least 1, got -103739.
  → Auto-calculated optimal ratio: 0.2624
  → Expected utilization: 95.0%
Error: max_tokens must be at least 1, got -8010.
  → Auto-calculated optimal ratio: 0.26

In [306]:
export_coca_markdown_blocks(analysis_result, test_keyword)


📄 Enhanced COCA forensic analysis exported: coca_forensic_vehicle_blocks.md (6160 chars)


'coca_forensic_vehicle_blocks.md'

## Major Questions Doctrine

In West Virginia v EPA, the Supreme Court invoked the "major questions doctrine" to limit the EPA's authority under the Clean Air Act. The Court ruled that significant regulatory actions require clear congressional authorization. This decision has implications for administrative law and the balance of power between agencies and Congress.

Justice Kagan mentions "get-out-of-text-free card" on page 28 (), with a footnote #8

> *8. "The majority opinion at least addresses the statute’s text, though overstating its ambiguity and approaching the action taken under it with unwarranted “skepticism.” Ante, at 28; see ante, at 28–31. The concurrence, by contrast, concludes that the Clean Air Act does not clearly enough authorize EPA’s Plan without ever citing the statutory text. See ante, at 13–16. **Nowhere will you find the concurrence ask: What does the phrase “best system of emission reduction” mean? §7411(a)(1). So much for “begin[ning], as we must, with a careful examination of the statutory text.”** Henson v. Santander Consumer USA Inc., 582 U. S. 79,___ (2017) (slip op., at 3)."*


From the Clean Air Act, here is the relevant text for `"best system of emission reduction"`:

> #### [42 U.S. Code § 7411 - Standards of performance for new stationary sources](https://www.law.cornell.edu/uscode/text/42/7411)
> (a)Definitions
> - For purposes of this section:
> - (1)The term “standard of performance” means a standard for emissions of air pollutants which reflects the degree of emission limitation achievable through the application of the best system of emission reduction which (taking into account the cost of achieving such reduction and any nonair quality health and environmental impact and energy requirements) the Administrator determines has been adequately demonstrated.

### Using COCA, we might attempt to answer Justice Kagan's question!

- Think about `best system` & `emission reduction` in context of COCA usage. In theory you could get AI Agent tools to give you a statistical linguistic answer based on COCA usage of these terms, and use a secondary agent to take the summaries and attempt to answer the question!

### JSON Mode Analysis

Now let's try the structured JSON mode for systematic data extraction:

In [69]:
# JSON mode analysis for structured data extraction
if 'bovine_kwic' in locals() and bovine_kwic:
    json_analysis = coca_forensic_tool._run(
        keyword=test_keyword,
        results_json=bovine_kwic,
        analysis_focus="semantic_variation",  # Try different focus
        max_contexts=50,
        return_json=True,  # Request structured JSON
        extraction_strategy="all",
        debug=False
    )
    
    print("📊 Structured JSON Analysis Results:")
    print("="*50)
    
    if isinstance(json_analysis, dict):
        for key, value in json_analysis.items():
            if key == 'reasoning_content' and isinstance(value, list):
                print(f"{key}: {len(value)} reasoning steps")
                for i, step in enumerate(value[:3], 1):  # Show first 3 steps
                    print(f"  {i}. {step[:100]}...")
            elif isinstance(value, str) and len(value) > 100:
                print(f"{key}: {value[:100]}...")
            else:
                print(f"{key}: {value}")
        
        # Export JSON analysis too
        export_coca_markdown(json_analysis, f"{test_keyword}_json")
    else:
        print("Unexpected result format:", type(json_analysis))
        print(json_analysis)
else:
    print("❌ No bovine_kwic data available for JSON analysis.")

❌ No bovine_kwic data available for JSON analysis.


### Multiple Analysis Focus Modes

The COCA forensic linguistics tool supports different analysis approaches:

- **`forensic_linguistics`**: Comprehensive forensic analysis including semantic range, register variation, collocational profiles
- **`semantic_variation`**: Focus on polysemy, meaning boundaries, metaphorical vs literal usage  
- **`register_analysis`**: Compare formal vs informal usage, genre-specific conventions
- **`diachronic`**: Analyze temporal patterns and semantic change (when time data available)
- **`comparative`**: Cross-genre comparative analysis

### Custom Analysis Example

Let's try a register analysis to see how the keyword varies across COCA genres:

In [27]:
# Register analysis example
if 'bovine_kwic' in locals() and bovine_kwic:
    register_analysis = coca_forensic_tool._run(
        keyword=test_keyword,
        results_json=bovine_kwic,
        analysis_focus="register_analysis",
        max_contexts=40,
        return_json=False,
        extraction_strategy="all"
    )
    
    print("📝 Register Analysis Results (first 800 chars):")
    print("="*60)
    print(register_analysis[:800])
    print("\n[Analysis continues...]")
    
    # Export this analysis
    export_coca_markdown(register_analysis, f"{test_keyword}_register")
    
else:
    print("❌ Need bovine_kwic data for register analysis")

🔬 COCA Analysis: 0 contexts across 248 genres | focus=register_analysis
📝 Register Analysis Results (first 800 chars):
[{'type': 'reasoning_content', 'reasoning_content': {'text': 'The user provides a request: "You are a computational forensic linguistics AI agent analyzing COCA data. ... Provide structured analysis ... Use ONLY the provided COCA contexts". However, the contexts list is empty: total contexts 0, and the KWIC contexts show none. So we need to handle that: no data available. We should explain that analysis cannot be performed because there\'s no data, but can discuss methodology and limitations. Provide structured sections as requested, but note emptiness.\n\nWe must comply with policy: no disallowed content. It\'s fine. Provide a thorough answer acknowledging lack of data and explaining what could be done if data were present, and the limitations.\n\nWe must not fabricate data. So provide analysis based on zero occurrences, which implies that the word "bovine" does not a

In [33]:
# Enhanced COCA markdown export function to handle structured block responses
def export_coca_markdown_blocks(result, keyword: str, filename: str = None):
    """
    Export COCA forensic linguistics analysis to markdown, handling both dict and list formats.
    
    This function can process:
    - Dictionary results (JSON mode)
    - List of blocks (typical LangChain/Bedrock response format)
    - Simple string results
    """
    import json
    from datetime import datetime
    
    def _sanitize(name: str) -> str:
        return ''.join(c if (c.isalnum() or c in ('-','_')) else '_' for c in name.strip()) or 'analysis'
    
    safe_keyword = _sanitize(keyword)
    outname = filename or f"coca_forensic_{safe_keyword}_blocks.md"
    
    lines = [f"# COCA Forensic Linguistics Analysis: {keyword}\n\n"]
    lines.append(f"*Generated: {datetime.utcnow().isoformat()}Z*\n\n")
    
    # Handle different result formats
    if isinstance(result, list):
        # Extract reasoning content and main text from blocks
        reasoning_parts = []
        main_text_parts = []
        
        for block in result:
            if isinstance(block, dict):
                # Check for reasoning content
                if block.get('type') == 'reasoning_content':
                    rc = block.get('reasoning_content', {})
                    if isinstance(rc, dict) and 'text' in rc:
                        reasoning_parts.append(rc['text'])
                    elif isinstance(rc, str):
                        reasoning_parts.append(rc)
                
                # Check for main text content
                if block.get('type') == 'text' and 'text' in block:
                    main_text_parts.append(block['text'])
                elif 'text' in block and block.get('type') != 'reasoning_content':
                    main_text_parts.append(block['text'])
            elif isinstance(block, str):
                main_text_parts.append(block)
        
        # Add reasoning framework if found
        if reasoning_parts:
            lines.append("## Methodological Framework\n\n")
            lines.append("```text\n")
            lines.append('\n\n'.join(reasoning_parts))
            lines.append("\n```\n\n")
        
        # Add main analysis
        if main_text_parts:
            lines.append("## Analysis\n\n")
            lines.append('\n\n'.join(main_text_parts))
            lines.append("\n\n")
    
    elif isinstance(result, dict):
        # Handle dictionary format (existing logic)
        reasoning = result.get('reasoning_content', [])
        if reasoning:
            lines.append("## Methodological Framework\n\n")
            lines.append("```text\n")
            if isinstance(reasoning, list):
                lines.append('\n'.join(str(r) for r in reasoning))
            else:
                lines.append(str(reasoning))
            lines.append("\n```\n\n")
        
        # Add structured sections
        sections = [
            ('semantic_analysis', 'Semantic Analysis'),
            ('register_patterns', 'Register and Genre Patterns'),
            ('forensic_implications', 'Forensic Linguistics Assessment'),
            ('summary', 'Summary'),
            ('limitations', 'Limitations')
        ]
        
        for field, title in sections:
            if field in result and result[field]:
                lines.append(f"## {title}\n\n")
                lines.append(f"{result[field]}\n\n")
        
        # Add distribution data if available
        if 'genre_distribution' in result:
            lines.append("## Corpus Distribution\n\n")
            lines.append("```json\n")
            lines.append(json.dumps(result['genre_distribution'], indent=2))
            lines.append("\n```\n\n")
    
    else:
        # Handle simple string format
        lines.append("## Analysis\n\n")
        lines.append(str(result))
        lines.append("\n\n")
    
    # Add metadata footer
    lines.append("---\n\n")
    lines.append(f"*Analysis completed using COCA Forensic Linguistics Tool*\n")
    lines.append(f"*Keyword: {keyword} | Export timestamp: {datetime.utcnow().isoformat()}Z*\n")
    
    content = ''.join(lines)
    
    with open(outname, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 Enhanced COCA forensic analysis exported: {outname} ({len(content)} chars)")
    return outname

# Re-export the register analysis with the enhanced function
if 'register_analysis' in locals():
    print("🔄 Re-exporting register analysis with enhanced block parser...")
    export_coca_markdown_blocks(register_analysis, f"{test_keyword}_register_fixed")
else:
    print("❌ No register_analysis variable found to re-export")

❌ No register_analysis variable found to re-export


In [30]:
# Re-export all previous analyses with the enhanced block parser
if 'analysis_result' in locals():
    print("🔄 Re-exporting main forensic linguistics analysis...")
    export_coca_markdown_blocks(analysis_result, f"{test_keyword}_forensic_fixed")

if 'json_analysis' in locals():
    print("🔄 Re-exporting JSON analysis...")
    export_coca_markdown_blocks(json_analysis, f"{test_keyword}_json_fixed")

print("\n✅ All COCA analyses re-exported with proper block formatting!")

🔄 Re-exporting main forensic linguistics analysis...
📄 Enhanced COCA forensic analysis exported: coca_forensic_best_system_forensic_fixed_blocks.md (19237 chars)

✅ All COCA analyses re-exported with proper block formatting!


In [ ]:
# Test the improved context extraction with balanced genre sampling
print(f"🔧 Testing improved COCA forensic linguistics analysis with genre-balanced sampling...")
    
test_analysis = coca_forensic_tool._run(
    keyword=test_keyword,
    results_json=bovine_kwic_json,  # Use the JSON data
    analysis_focus="forensic_linguistics",
    max_contexts=100,  # Limit to 100 for testing, but ensure balanced across genres
    return_json=False,
    extraction_strategy="all",
    debug=True  # Enable debug to see genre distribution
)

print(f"\n✅ Improved analysis complete!")
print(f"Result type: {type(test_analysis)}")

if isinstance(test_analysis, str):
    # Look for evidence that all genres are being recognized
    genre_mentions = {}
    for genre in ['acad', 'blog', 'fic', 'mag', 'news', 'spok', 'tvm', 'web']:
        if genre.lower() in test_analysis.lower():
            genre_mentions[genre] = test_analysis.lower().count(genre.lower())
    
    print(f"📊 Genre mentions in analysis: {genre_mentions}")
    print(f"First 800 characters:\n{test_analysis[:800]}...")
elif isinstance(test_analysis, list):
    print(f"Got {len(test_analysis)} result blocks")
    for i, block in enumerate(test_analysis[:2]):  # Show first 2 blocks
        print(f"Block {i+1}: {type(block)} - {str(block)[:100]}...")

# Export this improved analysis
export_coca_markdown_blocks(test_analysis, f"{test_keyword}_improved")

## Token Limit Handling

The improved COCA forensic linguistics tool now includes smart token limit detection and ratio suggestions. When your dataset is too large, it will:

1. **Estimate token usage** before sending to the model
2. **Calculate suggested ratios** for data reduction  
3. **Provide specific filtering recommendations**
4. **Handle AWS Bedrock token limit errors gracefully**

This prevents failed runs and gives you actionable steps to optimize your dataset size.

In [ ]:
# Test the token limit detection and ratio suggestion
print(f"🧪 Testing token limit handling with full 'dictionary' dataset...")
print(f"📊 Dataset size: {len(bovine_kwic_json)} genre_year combinations")

# This should trigger the token limit detection and provide a suggested ratio
large_dataset_result = coca_forensic_tool._run(
    keyword=test_keyword,
    results_json=bovine_kwic_json,  # Full dataset - likely to exceed token limits
    analysis_focus="forensic_linguistics",
    return_json=True,  # Get structured response with ratio info
    extraction_strategy="all",
    debug=True
)

print(f"\n🎯 Result type: {type(large_dataset_result)}")

if isinstance(large_dataset_result, dict) and "error" in large_dataset_result:
    print(f"✅ Token limit detection worked!")
    if "suggested_ratio" in large_dataset_result:
        ratio = large_dataset_result["suggested_ratio"]
        print(f"📏 Suggested ratio: {ratio:.3f}")
        print(f"💡 This means reduce your dataset to ~{ratio*100:.1f}% of current size")
    else:
        print("ℹ️ Error detected but no ratio calculated (likely AWS Bedrock specific error)")
elif isinstance(large_dataset_result, str) and "Token Limit" in large_dataset_result:
    print(f"✅ Token limit detection worked! (string response)")
else:
    print(f"🤔 Unexpected result - either dataset was small enough or error handling needs adjustment")
    if isinstance(large_dataset_result, dict):
        for key, value in large_dataset_result.items():
            print(f"  {key}: {str(value)[:100]}...")
    else:
        print(f"  Content preview: {str(large_dataset_result)[:200]}...")